In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
path = "/data/pcpe_01.csv"

In [4]:
import pandas as pd
import numpy as np
import keras
import keras.optimizers.adam
import tensorflow as tf

2024-05-08 14:27:06.517570: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
df = pd.read_csv(path, delimiter=';')

In [8]:
df['CNAB'].value_counts()

CNAB
213    24003
201    23420
117    13200
218    10994
105    10472
114    10095
209     8420
104     7459
120     6466
205     5028
112     4722
206     4498
102     2354
101     1775
106     1751
220      755
204      713
217      404
219      340
123      216
103      154
202      150
118      135
211      128
110      118
107      110
999       93
215       92
203       87
113       68
214       32
119       25
207       18
212        7
111        2
115        1
Name: count, dtype: int64

In [12]:
print(df.dtypes)

NUMERO_CASO             object
NUMERO_BANCO            object
NUMERO_AGENCIA           int64
NUMERO_CONTA             int64
TIPO                     int64
CPF_CNPJ_TITULAR        object
CNAB                     int64
CNAB=101                 int64
CNAB=119                 int64
CNAB=120                 int64
CNAB=123                 int64
CNAB=201                 int64
CNAB=202                 int64
CNAB=203                 int64
CNAB=204                 int64
CNAB=205                 int64
CNAB=206                 int64
CNAB=207                 int64
CNAB=209                 int64
CNAB=211                 int64
CNAB=212                 int64
CNAB=213                 int64
CNAB=214                 int64
CNAB=215                 int64
CNAB=217                 int64
CNAB=218                 int64
CNAB=219                 int64
CNAB=220                 int64
CNAB=999                 int64
DATA_LANCAMENTO         object
NUMERO_DOCUMENTO        object
VALOR_TRANSACAO        float64
NATUREZA

In [13]:
columns_num = [
    'DIA_LANCAMENTO', 
    'MES_LANCAMENTO',
    'VALOR_TRANSACAO',
    'VALOR_SALDO',
]

columns_cat = [
    'TIPO',
    'CNAB',
    'NATUREZA_SALDO'
]

target = ['I-a']
y = df[target].values
y = np.nan_to_num(y)


In [14]:
df[columns_num]

,DIA_LANCAMENTO,MES_LANCAMENTO,VALOR_TRANSACAO,VALOR_SALDO
0,1,1,0.15,86.76
1,1,1,0.40,87.16
2,2,1,0.06,718.27
3,2,1,0.02,718.29
4,2,1,6573.30,6573.30
...,...,...,...,...
138300,1,3,2.39,3.39
138301,1,3,2.39,1.00
138302,1,3,2.39,115.24
138303,1,3,0.00,1.88


In [15]:
df[columns_cat]

,TIPO,CNAB,NATUREZA_SALDO
0,2,218,C
1,2,218,C
2,2,211,C
3,2,211,C
4,1,201,C
...,...,...,...
138300,1,206,C
138301,1,102,C
138302,2,102,C
138303,1,205,C


In [16]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_num = scaler.fit_transform(df[columns_num])

X_num

array([[0.00000000e+00, 0.00000000e+00, 7.50000000e-08, 3.70279998e-05],
       [0.00000000e+00, 0.00000000e+00, 2.00000000e-07, 3.71987144e-05],
       [3.33333333e-02, 0.00000000e+00, 3.00000000e-08, 3.06547964e-04],
       ...,
       [0.00000000e+00, 1.81818182e-01, 1.19500000e-06, 4.91828803e-05],
       [0.00000000e+00, 1.81818182e-01, 0.00000000e+00, 8.02358685e-07],
       [0.00000000e+00, 1.81818182e-01, 0.00000000e+00, 8.02358685e-07]])

In [17]:
X = np.column_stack([X_num, df[columns_cat]])

In [21]:
mid = 60000
X_train = X[:mid]
y_train = y[:mid]
X_test = X[mid:138305]
y_test = y[mid:138305]

In [22]:
max_features = np.max(df['CNAB']) + 1
sequence_length = 10 
batch_size = 128

In [ ]:
data_train = tf.keras.utils.timeseries_dataset_from_array(
    X_train[:-sequence_length],
    y_train[sequence_length:],
    sequence_length=sequence_length,
    sequence_stride=1,
    sampling_rate=1,
    batch_size=batch_size,
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None,
)

In [ ]:
data_test = tf.keras.utils.timeseries_dataset_from_array(
    X_test[:-sequence_length],
    y_test[sequence_length:],
    sequence_length=sequence_length,
    sequence_stride=1,
    sampling_rate=1,
    batch_size=batch_size,
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None,
)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Embedding(max_features, batch_size, input_length=sequence_length),
    keras.layers.LSTM(16,  return_sequences=True, input_shape=[None, X.shape[-1]]),
    keras.layers.LSTM(16, ),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.build()
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc', tf.keras.metrics.AUC()])
model.summary()

In [ ]:
history = model.fit(data_train, epochs=10)

In [ ]:
y_pred = model.predict(data_test)